In [ ]:
#Team member(s): Jasleen and Samah

In [ ]:
!pip install transformers

In [ ]:
!pip install tensorflow_text

In [ ]:
!pip install -U tensorflow-estimator==2.10.0

In [ ]:
!pip install tf-models-official

In [ ]:
!pip3 uninstall keras
!pip3 install keras --upgrade

In [ ]:
import tensorflow as tf

In [ ]:
device_list = tf.test.gpu_device_name()

In [ ]:
device_list

In [ ]:
if device_list != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_list))

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import binary_focal_crossentropy
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel, AdamW, BertConfig
import torch
import matplotlib.pyplot as plt
import requests
import re
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, plot_confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
#load the data
Tweet = pd.read_csv('/Tweets.csv')
#Tweet.shape
Tweet = Tweet[['text','sentiment']]
#drop and check null values
Tweet = Tweet.dropna()
Tweet.isnull().values.any() 

In [ ]:
#Prepare training and testing data
text =Tweet['text']
label=Tweet['sentiment']

# convert label into 3 classes
lab = LabelBinarizer()
lab.fit(label)
labels = lab.transform(label)

In [ ]:
print(20*'--')
print('Total number of tweets = {}'.format(len(text)))
print('Split by sentiment')
print(Tweet['sentiment'].value_counts())

In [ ]:
x_data = text
y_data = labels

In [ ]:
# split dataset into training, validation, and testing
main_data,testing_data, main_label,testing_label = train_test_split(x_data, y_data, 
                                                                            test_size=0.1, 
                                                                            random_state=42, 
                                                                            shuffle=True, 
                                                                            stratify= y_data )
training_data,val_data, training_label,val_label = train_test_split(main_data, main_label, 
                                                                            test_size=0.1, 
                                                                            random_state=42, 
                                                                            shuffle=True, 
                                                                            stratify= main_label)

In [ ]:
#Team member(s): Samah

In [ ]:
# load encoder 
bert_encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
# load preprocessor
bert_preprocessor_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [ ]:
bert_preprocessor = hub.KerasLayer(bert_preprocessor_url)
bert_model = hub.KerasLayer(bert_encoder_url)

In [ ]:
# this function is used to understand bert output
def get_embedding_vector(tweets):

  #preprocessed_tweets returns input_id, attention_mask, and input_type_id
  preprocessed_tweets = bert_preprocessor(tweets)

  # encoded_tweets returns default, encoder_outputs, pooled_output, sequence_output
  #pooled_output: represents the embedding for the entire tweet
  bert_results = bert_model(preprocessed_tweets)
  
  return bert_results['pooled_output']


In [ ]:
# Check bert output on sample of training data
t = training_data[:5]
bert_output = get_embedding_vector(t)
bert_output

# MODEL 1

In [ ]:
#Team member(s): Samah

In [ ]:
#Bert_layers
# first layer takes input as a string
input_layer = tf.keras.Input(shape=(), dtype = tf.string, name = 'text')
# passed input_layer into bert_preprocessing and bert_model
preprocessed_text = bert_preprocessor(input_layer)
outputs = bert_model(preprocessed_text)
# extract bert output layer and pass it to the rest of the network
bert_outputs = outputs['pooled_output']
# we added dropout layer for optimization purposes 
x = tf.keras.layers.Dropout(0.2, name='dropout')(bert_outputs)
x = tf.keras.layers.Dense(128, activation='relu', name='dense')(x)
output = tf.keras.layers.Dense(3,activation='softmax', name='output')(x)

#final model
model = tf.keras.Model(inputs = [input_layer], outputs = [output])


In [ ]:
model.summary()

In [ ]:
#initializing optimizer, lost function, and mertics
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
#Training the first model 
history = model.fit(training_data, training_label, epochs=10, batch_size = 32, validation_data=(val_data, val_label))

In [ ]:
# Evaluate the model on testing dataset
model_eval= model.evaluate(testing_data, testing_label)

In [ ]:
# Predicting the sentiment of any text(we used testing dataset)
y_predict = model.predict(testing_data)
print(y_predict)
y_predict = np.argmax(y_predict, axis = 1)
testing_label= np.argmax(testing_label, axis = 1)

In [ ]:
#Team member(s): Jasleen and Samah

In [ ]:
#plotting confusion matrix
mul_c = confusion_matrix(testing_label, y_predict)

In [ ]:
ax = sns.heatmap(mul_c, annot=True, cmap='RdPu', fmt = 'g')
ax.set_title('Confusion Matrix\n\n');
ax.set_xlabel('\nPredicted ')
ax.set_ylabel('Actual');
# Ticket labels
ax.xaxis.set_ticklabels(['Negative','Neutral', 'Positive'])
ax.yaxis.set_ticklabels(['Negative','Neutral', 'Positive'])
# Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
print(classification_report(testing_label, y_predict))

#MODEL 2

In [ ]:
#Team member(s): Samah

In [ ]:
#MODEL 2:

#Bert_layers
# first layer takes input as a string
input_layer = tf.keras.Input(shape=(), dtype = tf.string, name = 'text')
# passed input_layer into bert_preprocessing and bert_model
preprocessed_text = bert_preprocessor(input_layer)
outputs = bert_model(preprocessed_text)
#extract bert output layer and pass it to the rest of the network
bert_outputs = outputs['pooled_output']
#reshape bert outputs for Bi-LSTM input
LSTM_input = tf.keras.layers.Reshape((1,768))(bert_outputs)
# Bidirectional LSTM layer 
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(15, input_shape=()), name = "Bi-LSTM")(LSTM_input)
# add dropout layer for optimization purposes
x = tf.keras.layers.Dropout(0.2, name='dropout')(x)
x = tf.keras.layers.Dense(128, activation='relu', name='dense')(x)
output = tf.keras.layers.Dense(3,activation='softmax', name='output')(x)

#final model
biLSTM_model = tf.keras.Model(inputs = [input_layer], outputs = [output])

In [ ]:
biLSTM_model.summary()

In [ ]:
#initializing optimizer, lost function, and mertics
biLSTM_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Training second model(LSTM Bi_Dirictional model)
history2 = biLSTM_model.fit(training_data, training_label, epochs=10, batch_size = 32, validation_data=(val_data, val_label))

In [ ]:
# Evaluate the model on testing dataset
biLSTM_model.evaluate(testing_data, testing_label)

In [ ]:
# Predicting the sentiment of any text (we used testing dataset)
y2_predict = biLSTM_model.predict(testing_data)
print(y2_predict)
y2_predict = np.argmax(y2_predict, axis = 1)
testing_label= np.argmax(testing_label, axis = 1)

In [ ]:
#Team member(s): Jasleen and Samah

In [ ]:
#confusion matrix for output
mul_c = confusion_matrix(testing_label, y2_predict)

In [ ]:
#plotting confusion matrix
ax = sns.heatmap(mul_c, annot=True, cmap='RdPu', fmt = 'g')
ax.set_title('Confusion Matrix\n\n');
ax.set_xlabel('\nPredicted ')
ax.set_ylabel('Actual');
# Ticket labels
ax.xaxis.set_ticklabels(['Negative','Neutral', 'Positive'])
ax.yaxis.set_ticklabels(['Negative','Neutral', 'Positive'])
# Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
print(classification_report(testing_label, y2_predict))

References:
1. https://medium.com/@claude.feldges/text-classification-with-tf-idf-lstm-bert-a-quantitative-comparison-b8409b556cb3
2. https://github.com/codebasics/deep-learning-keras-tf-tutorial/blob/master/47_BERT_text_classification/BERT_email_classification-handle-imbalance.ipynb
3. https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
